In [1]:
import covalent as ct
import pickle
import os
import pennylane.numpy as np
import torch
from itertools import cycle
import matplotlib.pyplot as plt
import pennylane as qml
from itertools import combinations
from pytorch_scipy_optimizer import Optimizer

# Need this to resolve paths pater
base_path = os.getcwd()

/Users/jbaker/opt/miniconda3/envs/algo-time-series/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@ct.electron
def load_data_from_pickle(path):
    data = pickle.load(open(path, 'rb'))
    return data

In [3]:
Xtr = load_data_from_pickle("".join([base_path, "/data/Xtr.pickle"])) # 3d array

In [4]:
@ct.electron
def data_santity_check(data):
    # Only accept 3d data. (num_series, num_features, num_time_points)
    X = np.asarray(data)
    # Assume 3d array is list of nd-feature time series
    if len(X.shape) != 3:
        raise TypeError("Dimensions of X must be 3 dimensional: (num_series, num_features, num_time_points)")

In [5]:
data_santity_check(Xtr)

In [55]:
@ct.electron
def get_series_training_dataloader(Xtr, n_series_batch, shuffle=True):
    dataloader_series = \
        torch.utils.data.DataLoader(Xtr, batch_size=n_series_batch, shuffle=shuffle)
    return dataloader_series

@ct.electron
def get_timepoint_training_dataloader(Xtr, n_t_batch, shuffle=True):
    n_time_points = Xtr.shape[2]
    data_loader_timeidxs = \
        torch.utils.data.DataLoader(torch.tensor(np.arange(n_time_points)), batch_size=n_t_batch, shuffle=True)
    return data_loader_timeidxs

In [7]:
x_cycler = get_series_training_dataloader(Xtr, 10)
t_cycler = get_timepoint_training_dataloader(Xtr, 10)

In [8]:
@ct.electron
def arctan_penalty(sigma, contraction_hyperparameter):
    prefac = 1/(np.pi)
    sum_terms = torch.arctan(2*np.pi*contraction_hyperparameter*torch.abs(sigma))
    mean = sum_terms.mean()
    return prefac*mean

In [9]:
sigma = torch.tensor([100, 100, 100])
pen = arctan_penalty(sigma, 1)

In [10]:
@ct.electron
def create_diagonal_circuit(D, n, k=None):
    if k is None:
        k = n
    cnt = 0
    for i in range(1, k + 1):
        for comb in combinations(range(n), i):
            if len(comb) == 1:
                qml.RZ(D[cnt], wires=[comb[0]])
                cnt += 1
            elif len(comb) > 1:
                cnots = [comb[i : i + 2] for i in range(len(comb) - 1)]
                for j in cnots:
                    qml.CNOT(wires=j)
                qml.RZ(D[cnt], wires=[comb[-1]])
                cnt += 1
                for j in cnots[::-1]:
                    qml.CNOT(wires=j)

In [11]:
D = torch.tensor([2, 2, 2])
n_qubits = 2
create_diagonal_circuit(D, n_qubits, 2)

In [24]:
@ct.electron
def get_device(n_qubits):
    device = qml.device('lightning.qubit', wires=n_qubits, shots=None)
    return device

In [25]:
device = get_device(n_qubits)

In [26]:
@ct.electron
@qml.qnode(device, interface='torch')
def get_anomaly_expec(x, t, D, alpha, wires, k, embed_func, transform_func, diag_func, observable,
                      embed_func_params={}, transform_func_params={}):
    embed_func(x, wires=wires, **embed_func_params)
    transform_func(alpha, wires, **transform_func_params)
    diag_func(D * t, n_qubits, k=k)
    qml.adjoint(transform_func)(alpha, wires=range(n_qubits), **transform_func_params
        )
    coeffs = np.ones(len(wires))/len(wires)
    H = qml.Hamiltonian(coeffs, observable)
    return qml.expval(H)

In [27]:
observable = [qml.PauliZ(i) for i in range(n_qubits)]
x = torch.tensor([0.1, 0.7])
t = torch.tensor([0.1])
D = torch.tensor([0.1, 0.1, 0.1])
k = 2
transform_func = qml.templates.StronglyEntanglingLayers
alpha_weights_shape = transform_func.shape(3, 2)
alpha = torch.tensor(np.random.uniform(0, 2 * np.pi, size=alpha_weights_shape), requires_grad=True
            ).type(torch.DoubleTensor)
embed_func = qml.templates.AngleEmbedding

get_anomaly_expec(x, t, D, alpha, range(n_qubits), k, embed_func, transform_func, create_diagonal_circuit, observable)

tensor(0.8834, dtype=torch.float64, grad_fn=<SqueezeBackward0>)

In [28]:
@ct.electron
def sample_D(sigma, mu):
    D = torch.normal(mu, sigma.abs())
    return D

In [29]:
D = sample_D(torch.tensor([1.0, 2.0, 3.0]), torch.tensor([0.0, 3.1, 4.2]))

In [30]:
@ct.electron
def get_single_point_cost(x, t, alpha, q, D_sample_func, sigma, mu, N_E, wires, k, embed_func, transform_func, diag_func, observable,
                      embed_func_params={}, transform_func_params={}):
    expecs = torch.zeros(N_E)
    for i in range(N_E):
        D = D_sample_func(sigma, mu)
        expec = get_anomaly_expec(x, t, D, alpha, wires, k, embed_func, transform_func, diag_func, observable,
                                   embed_func_params={}, transform_func_params={})
        expecs[i] = expec
    mean = expecs.mean()
    single_point_cost = (q - mean)**2/4
    return single_point_cost

In [31]:
q = torch.tensor([-1])
mu = torch.tensor([0.1, 0.2, 0.1])
sigma = torch.tensor([0.6, 4.1, 2.0])
N_E = 10
wires = range(n_qubits)
diag_func = create_diagonal_circuit
get_single_point_cost(x, t, alpha, q, sample_D, sigma, mu, N_E, wires, k, embed_func, transform_func, diag_func, observable,)

tensor([0.8288], grad_fn=<DivBackward0>)

In [32]:
@ct.electron
def get_time_series_cost(xt, alpha, q, D_sample_func, sigma, mu, N_E, wires, k, embed_func,
                         transform_func, diag_func, observable, t_cycler, embed_func_params={},
                         transform_func_params={}):
    if t_cycler is None:
        t_idxs = np.arange(xt.shape[1])
        ts = np.linspace(0.1, 2 * np.pi, xt.shape[1], endpoint=True)
    else:
        t_idxs = next(t_cycler)
        ts = np.linspace(0.1, 2 * np.pi, xt.shape[1], endpoint=True)[t_idxs]
    xt_batch = xt[:, t_idxs]
    xfunct = zip(xt_batch.T, ts)
    a_func_t = \
        [get_single_point_cost(x, t, alpha, q, sample_D, sigma, mu, N_E, wires,
                               k, embed_func, transform_func, diag_func, observable) for x, t in xfunct]
    single_time_series_cost = torch.tensor(a_func_t, requires_grad=True).mean()
    return single_time_series_cost

In [33]:
xt = Xtr[0]
t_cycler = get_timepoint_training_dataloader(Xtr, 10)
single_time_series_cost = \
     get_time_series_cost(xt, alpha, q, sample_D, sigma, mu, N_E, wires, k, embed_func,
                         transform_func, diag_func, observable, t_cycler, embed_func_params={},
                         transform_func_params={})

In [34]:
@ct.electron
def get_loss(alpha, q, D_sample_func, sigma, mu, N_E, wires, k, embed_func,
            transform_func, diag_func, observable, t_cycler, x_cycler, penalty, tau, embed_func_params={},
            transform_func_params={}):

    X_batch = next(x_cycler)
    single_costs = torch.zeros(X_batch.shape[0])
    for i, xt in enumerate(X_batch):
        single_time_series_cost = \
            get_time_series_cost(xt, alpha, q, sample_D, sigma, mu, N_E, wires, k, embed_func,
                                 transform_func, diag_func, observable, t_cycler, embed_func_params={},
                                 transform_func_params={})
        single_costs[i] = single_time_series_cost
    loss = 0.5*single_costs.mean() + penalty(sigma, tau)
    return loss

In [35]:
penalty = arctan_penalty
tau = 1
loss = get_loss(alpha, q, sample_D, sigma, mu, N_E, wires, k, embed_func,
            transform_func, diag_func, observable, t_cycler, x_cycler, penalty, tau)

In [36]:
@ct.electron
def get_initial_parameters(transform_func, n_qubits, transform_func_layers):
    initial_alpha =\
         torch.tensor(np.random.uniform(0, 2 * np.pi, size=transform_func.shape(transform_func_layers, n_qubits)),
         requires_grad=True).type(torch.DoubleTensor)
    initial_mu = torch.tensor(
                np.random.uniform(0, 2 * np.pi, 2 ** (n_qubits) - 1)).type(torch.DoubleTensor)
    initial_sigma = torch.tensor(
                np.random.uniform(0, 2 * np.pi, 2 ** (n_qubits) - 1)).type(torch.DoubleTensor)

    initial_q = torch.tensor(np.random.uniform(-1, 1)).type(torch.DoubleTensor)
    init_parameters = {'alpha': initial_alpha, 'mu': initial_mu, 'sigma': initial_sigma, 'q': initial_q}
    return init_parameters

In [37]:
transform_func = qml.templates.StronglyEntanglingLayers
n_qubits = 2
transform_func_layers = 3
init_parameters = get_initial_parameters(transform_func, n_qubits, transform_func_layers)
print(init_parameters)

{'alpha': tensor([[[2.6582, 4.0949, 2.5982],
         [3.1192, 4.8353, 3.2766]],

        [[0.2184, 3.6168, 4.4182],
         [0.9721, 0.4044, 5.0335]],

        [[2.2754, 3.4881, 0.4931],
         [2.4994, 3.5556, 0.5866]]], dtype=torch.float64, requires_grad=True), 'mu': tensor([5.9833, 3.9248, 0.2735], dtype=torch.float64), 'sigma': tensor([3.7458, 0.5436, 1.4291], dtype=torch.float64), 'q': tensor(-0.7046, dtype=torch.float64)}


In [38]:
@ct.electron
def train_model(alpha, q, D_sample_func, sigma, mu, N_E, wires, k, embed_func, n_qubits,
                transform_func, diag_func, observable, t_cycler, x_cycler, penalty, tau,
                optimizer_params, initial_parameters):
    f = lambda alpha, mu, sigma, q: get_loss(alpha=alpha, mu=mu, sigma=sigma, q=q,
                                             D_sample_func=D_sample_func, N_E=N_E, wires=range(n_qubits),
                                             k=k, embed_func=embed_func, transform_func=transform_func,
                                             diag_func=diag_func, observable=observable,
                                             t_cycler=t_cycler, x_cycler=x_cycler,
                                             penalty=penalty, tau=tau)
    optimizer = Optimizer(f, initial_parameters, optimizer_params)
    opt_params = optimizer.optimize()
    return {"opt_params": opt_params}

In [39]:
initial_parameters = get_initial_parameters(transform_func, n_qubits, transform_func_layers)
optimizer_params = {
                "method": "COBYLA",
                "options": {"disp": True, "maxiter": 1},
                "jac": False,
            }
opt_params =\
    train_model(alpha, q, sample_D, sigma, mu, N_E, wires, k, embed_func, n_qubits,
                transform_func, diag_func, observable, t_cycler, x_cycler, penalty, tau,
                optimizer_params, initial_parameters)
print(opt_params)        


   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    1   F = 5.150300E-01    MAXCV = 0.000000E+00
   X = 6.689771E-01   1.275315E+00   9.910960E-01   5.463059E+00   3.674307E+00
       3.371600E+00   6.075654E+00   9.679636E-01   4.447453E-01   3.816727E+00
       4.746373E-01   4.670682E+00   4.289763E+00   2.150155E+00   2.918334E+00
       5.595792E+00   4.015374E+00   2.052411E+00   5.432162E+00   3.549622E+00
       1.667371E+00   5.858200E+00   1.560359E+00   2.256683E+00   6.281301E-01
{'opt_params': {'alpha': tensor([[[0.6690, 1.2753, 0.9911],
         [5.4631, 3.6743, 3.3716]],

        [[6.0757, 0.9680, 0.4447],
         [3.8167, 0.4746, 4.6707]],

        [[4.2898, 2.1502, 2.9183],
         [5.5958, 4.0154, 2.0524]]], dtype=torch.float64, requires_grad=True), 'mu': tensor([5.4322, 3.5496, 1.6674], dtype=torch.float64), 'sigma': tensor([5.8582, 1.5604, 2.2567], dtype=torch.float64), 'q': tensor(0.6281, dtype=torch.float64)}}


In [40]:
opt_params['opt_params']

{'alpha': tensor([[[0.6690, 1.2753, 0.9911],
          [5.4631, 3.6743, 3.3716]],
 
         [[6.0757, 0.9680, 0.4447],
          [3.8167, 0.4746, 4.6707]],
 
         [[4.2898, 2.1502, 2.9183],
          [5.5958, 4.0154, 2.0524]]], dtype=torch.float64, requires_grad=True),
 'mu': tensor([5.4322, 3.5496, 1.6674], dtype=torch.float64),
 'sigma': tensor([5.8582, 1.5604, 2.2567], dtype=torch.float64),
 'q': tensor(0.6281, dtype=torch.float64)}

In [41]:
@ct.electron
def get_anomaly_scores(Xte, normal_cost, penalty, tau, results_dict, D_sample_func, wires, k, embed_func,
                       transform_func, diag_func, observable, get_time_resolved=False):
    opt_params = results_dict['opt_params']
    alpha = opt_params['alpha']
    mu = opt_params['mu']
    sigma = opt_params['sigma']
    q = opt_params['q']
    scores = torch.zeros(Xte.shape[0])
    for i, yt in enumerate(Xte):
        single_time_series_cost =\
            get_time_series_cost(yt, alpha, q, D_sample_func, sigma, mu, N_E, wires, k, embed_func,
                         transform_func, diag_func, observable, t_cycler=None)
        scores[i] = (normal_cost - single_time_series_cost - penalty(sigma, tau)).abs()
    return scores

In [42]:
Yte = load_data_from_pickle("".join([base_path, "/data/Xte_dirty_usdt_pos.pickle"]))
Yte = Yte[:, :, ::4]
normal_cost = 0
D_sample_func = sample_D
k = 2
n_qubits = 2
transform_func = qml.templates.StronglyEntanglingLayers
diag_func = create_diagonal_circuit
wires = range(n_qubits)
scores = get_anomaly_scores(Yte, normal_cost, penalty, tau, opt_params, D_sample_func, wires, k, embed_func,
                            transform_func, diag_func, observable, get_time_resolved=False)

done 0
done 1
done 2
done 3
done 4
done 5
done 6
done 7
done 8
done 9
done 10
done 11
done 12
done 13
done 14
done 15
done 16
done 17
done 18
done 19
done 20
done 21
done 22
done 23
done 24
done 25
done 26
done 27
done 28
done 29
done 30
done 31
done 32
done 33
done 34
done 35
done 36
done 37
done 38
done 39
done 40
done 41
done 42
done 43
done 44
done 45
done 46
done 47
done 48
done 49
done 50
done 51
done 52
done 53
done 54
done 55
done 56
done 57
done 58
done 59


In [43]:
print(scores)

tensor([0.5463, 0.5453, 0.5519, 0.5513, 0.5511, 0.5550, 0.5504, 0.5424, 0.5447,
        0.5494, 0.5538, 0.5511, 0.5338, 0.5447, 0.5448, 0.5509, 0.5481, 0.5448,
        0.5456, 0.5381, 0.5471, 0.5554, 0.5487, 0.5499, 0.5474, 0.5495, 0.5467,
        0.5494, 0.5497, 0.5477, 0.5500, 0.5525, 0.5551, 0.5479, 0.5517, 0.5512,
        0.5426, 0.5491, 0.5670, 0.5452, 0.5486, 0.5537, 0.5500, 0.5474, 0.5535,
        0.5459, 0.5499, 0.5487, 0.5445, 0.5491, 0.5644, 0.5460, 0.5480, 0.5540,
        0.5515, 0.5490, 0.5531, 0.5443, 0.5452, 0.5528], grad_fn=<CopySlices>)


In [68]:
@ct.electron
def get_transform_func(type):
    if type == 'sel':
        transform_func = qml.StronglyEntanglingLayers
    return transform_func

@ct.electron
def get_embedding_func(type):
    if type == 'ry':
        embed_func = qml.templates.AngleEmbedding
    return embed_func

## Electrons all work locally, let's dispatch them at lattices...

In [74]:
@ct.lattice
def training_workflow(Xtr_path, n_series_batch, n_t_batch, transform_func_type, n_qubits, transform_func_layers, transform_func):
    # load training data
    Xtr = load_data_from_pickle(Xtr_path)

    # check dimensions of input data are correct
    data_santity_check(Xtr)

    # Get dataloaders for series and time-point batches
    series_dataloader = get_series_training_dataloader(Xtr, n_series_batch)
    time_dataloader = get_timepoint_training_dataloader(Xtr, n_t_batch)

    # get penalty function
    pen = arctan_penalty

    # get transform_func
    transform_func = get_transform_func(transform_func_type)

    # get embedding func
    embed_func = get_embedding_func

    # get random initial parameters
    init_parameters = get_initial_parameters(transform_func, n_qubits, transform_func_layers)

    # get 
    
    return pen

In [75]:
Xtr_path = "".join([base_path, "/data/Xtr.pickle"])

dispatch_id = ct.dispatch(training_workflow)(Xtr_path, 10, 10, 'sel', 2, 3, qml.templates.StronglyEntanglingLayers)

ct_results = ct.get_result(dispatch_id=dispatch_id, wait=True)
Xtr = ct_results.result

TypeError: Object of type property is not JSON serializable

In [77]:
training_workflow.cova_imports

{'ct', 'electron'}

In [57]:
x = training_workflow(Xtr_path, 10, 10)